In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.conv_learner import *

In [3]:
PATH = 'data/dogbreed/'

In [4]:
ls {PATH}

labels.csv*  sample_submission.csv*  test/  train/


In [5]:
from fastai.plots import *

In [6]:
f_model = resnet34

In [7]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n)

In [11]:
print(val_idxs[:10])
print(n)

[2882 4514 7717   33 4924  532 8028 5860 6054 1543]
10222


In [12]:
def get_data(sz):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    return ImageClassifierData.from_csv(PATH, 'train', label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name='test')

In [13]:
sz=64

In [17]:
fnames,y,classes = csv_source(PATH, label_csv, skip_header=True, suffix='.jpg', continuous=False)
print(fnames[:4])
print(y[:4])
print(classes[:4])

['data/dogbreed/000bec180eb18c7604dcecc8fe0dba07.jpg', 'data/dogbreed/001513dfcb2ffafc82cccf4d8bbaba97.jpg', 'data/dogbreed/001cdf01b096e06d78e9e5112d419397.jpg', 'data/dogbreed/00214f311d5d2247d5dfe4fe24b2303d.jpg']
[19 37 85 15]
['affenpinscher', 'afghan_hound', 'african_hunting_dog', 'airedale']


In [14]:
data = get_data(sz)

IndexError: tuple index out of range

In [ ]:
data = data.resize(int(sz*1.3), 'tmp')

In [ ]:
learn = ConvLearner.pretrained(f_model, data)

In [ ]:
lrf=learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 0.08

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
lrs = np.array([lr/9,lr/3,lr])

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.save(f'{sz}')

In [ ]:
learn.sched.plot_loss()

In [ ]:
sz=128

In [ ]:
learn.set_data(get_data(sz))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
learn.save(f'{sz}')

In [ ]:
sz=256

In [ ]:
learn.set_data(get_data(sz))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
learn.save(f'{sz}')

In [ ]:
learn.load(f'{sz}')
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y)

## Confusion Matrix

In [ ]:
preds = np.argmax(probs, axis=1)
probs = probs[:,1]
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)
plot_confusion_matrix(cm, data.classes)

## Visualization